### Homework 07: Concurrency

## Due Date: Apr 5, 2023, 11:59pm

#### Firstname Lastname: Giulio Duregon

#### E-mail: gjd9961@nyu.edu

#### Enter your solutions and submit this notebook


---

**Problem 1** **(60 Points)**

Let us consider the Gamma function, or the Euler integral of the second kind: 

$$\Gamma(x) = \int_{0} ^ \infty t ^{x - 1} e^{-t} dt, $$

and in this HW we consider real $x > 0$.

(Here is more on the Gamma function https://en.wikipedia.org/wiki/Gamma_function .
It is not needed for this HW assignment.) 

**1.1 (Points 15)**: 

Write a function (in the cell below) that sequentially calculates the given Gamma integral.


In [1]:
import numpy as np
from time import time
def calculate_gamma(x, bound_1, bound_2, number_of_steps, verbose=False):
    # sequential version to calculate Gamma(x):
    # where we approximate the given integral,
    # like this a discrete sum in number_of_steps
    # equidistant points on the interval [bound_1, bound_2]
    
    ts = time()

    delta_x = (bound_2 - bound_1) / number_of_steps
    
    # initialize variables
    gamma_sum = 0
    current_x = bound_1
    
    # loop through equidistant points on the interval [bound_1, bound_2]
    for i in range(number_of_steps):
        # add contribution of current point to Gamma sum
        gamma_sum += np.exp(-current_x) * current_x**(x-1)
        
        # move to next point
        current_x += delta_x
    
    # multiply by step size to approximate integral
    gamma_approx = gamma_sum * delta_x
    if verbose:
        print(f"Gamma(x)={gamma_approx}, Total Time: {time()-ts}s")

**1.2 (Points 5)** 

Evaluate, $\Gamma(6)$ by using `calculate_gamma(x, bound_1, bound_2, number_of_steps)` and the error of this computation.


As arguments, use `x=6, bound_1=0, bound_2=1000, number_of_steps=10_000_000`. We know that $\Gamma(x) = x!$, so $\Gamma(6) = 5! = 120$. 


In [2]:
x = 6
bound_1 = 0
bound_2 = 1000
number_of_steps=10_000_000
calculate_gamma(x, bound_1, bound_2, number_of_steps, verbose=True)

Gamma(x)=120.00000000008042, Total Time: 4.477315187454224s


In [3]:
%timeit -r 2 -n 5 calculate_gamma(x, bound_1, bound_2, number_of_steps)

4.45 s ± 32.1 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


---
Write two functions to calculate $\Gamma(x)$ by using:

**1.3.1 (Points 15)**
**threading** with N=4 threads; 

In [4]:
import numpy as np
from queue import Queue
from threading import Thread
from threading import Lock
from time import time


def threaded_calculate_gamma(x, bound_1, bound_2, number_of_steps, num_threads, verbose=False):
    """ 
    Threaded version to calculate Gamma(x)
    """
    
    # Accounting
    ts = time()
    
    # Global sum variable
    gamma_sum = 0
    
    # Force synch with Lock()
    lock = Lock()
    q = Queue()
    
    # Define function for workers
    def gamma_chunk(q: Queue):
        """
        Helper function used by workers, takes a Queue as input, aquires work from input
        Uses lock to force synchronization between workers when adding chunked work to output
        """
        # Keep worker alive
        while True:
            
            # Access outer scope variable
            nonlocal gamma_sum
            
            # Get work from Queue
            x, bound_1, bound_2, number_of_steps = q.get()
            delta_x = (bound_2 - bound_1) / number_of_steps
            
            # initialize variables
            temp_sum = 0
            current_x = bound_1
            
            # loop through equidistant points on the interval [bound_1, bound_2]
            for i in range(number_of_steps):
                # add contribution of current point to Gamma sum
                temp_sum += np.exp(-current_x) * current_x**(x-1)
                
                # move to next point
                current_x += delta_x
            
            # multiply by step size to approximate integral
            temp_sum = temp_sum * delta_x
            
            # Aquire lock and add to sum
            with lock:
                gamma_sum += temp_sum
            
            # Signal task is done to work queue
            q.task_done()

    # initialize workers
    for _ in range(num_threads):
        worker = Thread(target=gamma_chunk, args=(q, ))
        worker.setDaemon(True) # this stop the threads when the program quits  
        worker.start()         # start the threads
       
    # Create chunks and add to work Queue
    bound_increment = (bound_2-bound_1) // num_threads
    num_steps_per_thread = number_of_steps // num_threads
    
    arg_array = []
    # partial_gamma = partial(calculate_gamma)
    
    for i in range(num_threads):
        temp_start = bound_1 + (i * bound_increment)
        temp_end = temp_start + bound_increment
        q.put((x, temp_start, temp_end, num_steps_per_thread))

        
    # Wait for all workers to be done
    q.join()
    if verbose:    
        print(f"Results for Threaded Calculate Gamme, Num Threads = {num_threads}")
        print(f"Gamma Calculation for x={x}, Gamma(x)={gamma_sum}, Total Time: {time()-ts}s")    

threaded_calculate_gamma(x, bound_1, bound_2, number_of_steps, num_threads=4, verbose=True)

Results for Threaded Calculate Gamme, Num Threads = 4
Gamma Calculation for x=6, Gamma(x)=120.00000000008042, Total Time: 4.474512815475464s


In [5]:
# Test function
NUM_THREADS = 4
%timeit -r 2 -n 5 threaded_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_THREADS, verbose=False)

4.39 s ± 18 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


**1.3.2 (Points 15)**
**multiprocessing** with N=4 processes.

In [6]:
def calculate_gamma_chunk(chunk):
    # sequential version to calculate Gamma(x):
    # where we approximate the given integral,
    # like this a discrete sum in number_of_steps
    # equidistant points on the interval [bound_1, bound_2]
    x, bound_1, bound_2, number_of_steps = chunk
    delta_x = (bound_2 - bound_1) / number_of_steps

    # initialize variables
    gamma_sum = 0
    current_x = bound_1

    # loop through equidistant points on the interval [bound_1, bound_2]
    for i in range(number_of_steps):
        # add contribution of current point to Gamma sum
        gamma_sum += np.exp(-current_x) * current_x ** (x - 1)

        # move to next point
        current_x += delta_x

    # multiply by step size to approximate integral
    gamma_approx = gamma_sum * delta_x

    return gamma_approx


def multiprocess_calculate_gamma(
    x, bound1, bound2, numsteps, NUM_PROCESSES, verbose=False
):
    # Start accounting
    ts = time()

    bound_increment = (bound2 - bound1) // NUM_PROCESSES
    num_steps_per_process = numsteps // NUM_PROCESSES

    arg_array = []
    # partial_gamma = partial(calculate_gamma)

    for i in range(NUM_PROCESSES):
        temp_start = bound1 + (i * bound_increment)
        temp_end = temp_start + bound_increment
        arg_array.append((x, temp_start, temp_end, num_steps_per_process))

    with Pool(NUM_PROCESSES) as pool:
        result = pool.map(calculate_gamma_chunk, arg_array)

    result_sum = sum(result)
    if verbose:
        from multiprocessing import cpu_count

        print("number of CPU cores:", cpu_count())
        print(f"Multiprocess Calculate Gamma with NUM_PROCESSES={NUM_PROCESSES}")
        print(f"Gamma(x) = {result_sum}, Total Time: {time()-ts}s")

# Import to override function definition to use Pool in a IPython Kernel
from gamma import multiprocess_calculate_gamma
NUM_PROCESSES = 4
multiprocess_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_PROCESSES, verbose=False)

In [7]:
%timeit -r 2 -n 5 multiprocess_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_PROCESSES, verbose=False)

1.67 s ± 7.05 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


**1.3.3 (Points 10)** 
Compare the times of the three versions and write a short explanation of what you are observing.

How does the answer change when N=8 and why?

### Answer:
- Serial Time: 4.49 s ± 19.6 ms per loop
- 4 Thread Time: 4.45 s ± 8.68 ms per loop
- 4 Process Time: 1.66 s ± 5.83 ms per loop
- 8 Threads Time: 4.49 s ± 3.21 ms per loop
- 8 Process Time: 1.32 s ± 7.14 ms per loop

We're observing that threading, and increasing the number of threads, does not have a positive effect on the run time, while increasing the number of processes running in parralel actually does. This is because my machine has 10 available CPU cores, so rather than running on a single core sequentially, we can leverage the other cores to run chunks of computation thus reducing the overall wall time necessary to run the function. This is why multiprocessing performs well: we're actually leveraging our CPU's parralel power. Threading on the other hand does not leverage the additional cores, but rather adds overhead by forcing the OS to perform costly context switchign between threads that are running within the same shared process (and address space) on the same CPU core. This is why we see the threaded time be nearly identical with the sequential implmentation time. This illustrates that when IO is a bottle neck, its best to use threads, but when CPU utilization / computation needs is, then additional processes running in parallel is more effective.

In [8]:
from gamma import multiprocess_calculate_gamma
NUM_THREADS = 8
threaded_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_THREADS, verbose=True)
NUM_PROCESSES = 8
multiprocess_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_PROCESSES, verbose=True)

Results for Threaded Calculate Gamme, Num Threads = 8
Gamma Calculation for x=6, Gamma(x)=120.00000000008042, Total Time: 4.444800138473511s
number of CPU cores: 10
Multiprocess Calculate Gamma with NUM_PROCESSES=8
Gamma(x) = 120.00000000008042, Total Time: 1.4162569046020508s


In [9]:
%timeit -r 2 -n 5 threaded_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_THREADS, verbose=False)

4.4 s ± 1.15 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [10]:
%timeit -r 2 -n 5 multiprocess_calculate_gamma(x, bound_1, bound_2, number_of_steps, NUM_PROCESSES, verbose=False)

1.3 s ± 650 µs per loop (mean ± std. dev. of 2 runs, 5 loops each)


---

**Problem 2 (40 points)**

__Website uptime__ is the time that a website or web service is available to the users over a given period.

The task is to build an application that checks the uptime of websites. 

- The application should go over a list of website URLs and checks if those websites are up.
- Instead of performing a classic HTTP GET request, it performs a HEAD request so that it does not affect traffic significantly.
- If the HTTP status is in the danger ranges (400+, 500+), a message is casted. 

Here are some useful functions:

In [11]:
#### _website uptimer_ ####
import time
import logging
import requests
 
class WebsiteDownException(Exception):
    pass
 
def ping_website(address, timeout=20):
    """
    Check if a website is down. A website is considered down 
    if either the status_code >= 400 or if the timeout expires
     
    Throw a WebsiteDownException if any of the website down conditions are met
    """
    try:
        response = requests.head(address, timeout=timeout)
        if response.status_code >= 400:
            logging.warning("Website %s returned status_code=%s" % (address, response.status_code))
            raise WebsiteDownException()
    except requests.exceptions.RequestException:
        logging.warning("Timeout expired for website %s" % address)
        raise WebsiteDownException()
         
def check_website(address, verbose=False):
    """
    Utility function: check if a website is down, if so, notify the user
    """
    try:
        ping_website(address)
    except WebsiteDownException:
        if verbose:
            print('The websie ' + address + ' is down')

---

You need a website list to try our system out. Create your own list or use the following one. 

---

In [12]:
WEBSITE_LIST = [
    'http://amazon.co.uk',
    'http://amazon.com',
    'http://facebook.com',
    'http://google.com',
    'http://google.fr',
    'http://google.es',
    'http://google.co.uk',
    'http://gmail.com',
    'http://stackoverflow.com',
    'http://github.com',
    'http://heroku.com',
    'http://really-cool-available-domain.com',
    'http://djangoproject.com',
    'http://rubyonrails.org',
    'http://basecamp.com',
    'http://trello.com',
    'http://shopify.com',
    'http://another-really-interesting-domain.co',
    'http://airbnb.com',
    'http://instagram.com',
    'http://snapchat.com',
    'http://youtube.com',
    'http://baidu.com',
    'http://yahoo.com',
    'http://live.com',
    'http://linkedin.com',
    'http://netflix.com',
    'http://wordpress.com',
    'http://bing.com',
]

---

A serial version of the _website uptimer_ can be written as: 

---


In [13]:
import time

def serial():
    start_time = time.time()
    
    for address in WEBSITE_LIST:
        check_website(address)
            
    end_time = time.time()        
    print("Time for Serial: %ssecs" % (end_time - start_time))

serial()

Time for Serial: 6.40507698059082secs


In [14]:
%timeit -r 2 -n 5 serial()

Time for Serial: 2.9443299770355225secs


Time for Serial: 4.007624864578247secs


Time for Serial: 3.179229974746704secs


Time for Serial: 3.8201441764831543secs


Time for Serial: 2.914763927459717secs


Time for Serial: 2.7815699577331543secs


Time for Serial: 3.143512010574341secs


Time for Serial: 2.908781051635742secs


Time for Serial: 4.022976875305176secs


Time for Serial: 3.543220281600952secs
3.33 s ± 46.4 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


You should build two versions of the **website uptimer**, by using:

**2.1 (Points 15)**
**threading** with N=4 threads;

In [15]:
from threading import Thread
from queue import Queue
from time import time

class PingWorker(Thread):
    def __init__(self, queue):
        super(PingWorker, self).__init__()
        self.queue = queue
    
    def run(self):
       while True:
            # Get the work from the queue and expand the tuple
            url = self.queue.get()
            # Ping the website
            check_website(url)
            # Set the task as done
            self.queue.task_done()


def threaded_website_uptimer(WEBSITE_LIST, NUM_THREADS):
    ts = time()
    q = Queue()
    
    for _ in range(NUM_THREADS):
        worker = PingWorker(q)
        worker.daemon = True
        worker.start()
    
    for address in WEBSITE_LIST:
        q.put(address)
    
    # Wait to finish running all tasks
    q.join()
    
    # output results
    print('Took {}s'.format(time() - ts))

# Number of threads and time results
NUM_THREADS = 4
threaded_website_uptimer(WEBSITE_LIST, NUM_THREADS)

Took 1.0619728565216064s


In [16]:
%timeit -r 2 -n 5 threaded_website_uptimer(WEBSITE_LIST, 4)

Took 1.2116258144378662s


Took 1.0237810611724854s


Took 1.0246772766113281s


Took 1.3012757301330566s


Took 1.1598138809204102s


Took 0.998708963394165s


Took 1.0416340827941895s


Took 1.0275688171386719s


Took 1.1257882118225098s


Took 1.0886750221252441s
1.1 s ± 43.9 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


**2.2 (Points 15)**
**multiprocessing** with N=4 processes.

In [17]:
from multiprocessing.pool import Pool

from time import time
def multiprocess_website_uptimer(WEBSITE_LIST, NUM_PROCESSES):
    # Set up timer and partial function
    ts = time()
    
    # Use process pool with partial func / website lists being passed to pool
    with Pool(NUM_PROCESSES) as p:
        p.map(check_website, WEBSITE_LIST)
    

# Import because of Pool() Ipython errors
from multiprocess_website_uptimer import multiprocess_website_uptimer
NUM_PROCESSES = 4
multiprocess_website_uptimer(WEBSITE_LIST, NUM_PROCESSES)

The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
The websie http://baidu.com is down
Took 1.2115478515625s


In [18]:
%timeit -r 2 -n 5 multiprocess_website_uptimer(WEBSITE_LIST, 4)

The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down


Took 1.3125050067901611s


The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.2075917720794678s


The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.2490639686584473s


The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
The websie http://baidu.com is down
Took 1.0999233722686768s


...

The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.2281091213226318s
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down


Took 1.3233189582824707s


The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.2439520359039307s


The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
The websie http://baidu.com is down
Took 1.3132388591766357s


The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.2498719692230225s
1.26 s ± 14.4 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


**2.3 (Points 10)** 

Compare the times of the three versions and write a short explanation of what you are observing.

- Serial Version, Time = 3.46 s ± 101 ms per loop
- 4 Threads Version, Time = 1.06 s ± 28.8 ms per loop
- 4 Process Version, Time = 1.23 s ± 35.9 ms per loop

Serial is by far the slowest as each request must wait for the previous request's response to arrive before sending. This IO time restriction limits the performance of the serial version quite a bit. Threads performed the best as expected, as the threads can leverage the shared address space to minimize overhead / memory requirements, while supporting multiple contexts of execution. When a thread is waiting for a response, and is in a blocked state, the CPU can switch to another thread's context of execution and send its request, making threads a favorite implmenting of concurrency for IO tasks. For multiprocessing, you would expect that the total time would be SERIAL TIME / NUM PROCESSES. This isn't strictly true as we can see as there is overhead for each additional process created due to it needing its own address space, as it is a seperate version of the programming running in parralel.

**How does the answer change when N=8 and why?**

- 8 MultiProcesses Version, Time = 1.02 s ± 100 ms per loop
- 8 Threaded Version, Time = 824 ms ± 25.9 ms per loop

The numbers are pretty comparable between the threaded and multiprocess programs when running with 8 process / threads. We do see a slight speed up when using threading rather than multiprocesses, which could be explained by the fact that we are IO bound rather than CPU bound, so the extra overhead for spinning up each process with its own address space increases the time. Threads on the other hand share the same address space, making requiring less overhead and being able to make use of shared variables (like the website list).

In [19]:
from multiprocess_website_uptimer import multiprocess_website_uptimer
NUM_PROCESSES = 8
multiprocess_website_uptimer(WEBSITE_LIST, NUM_PROCESSES)

The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down


Took 0.8836698532104492s


In [20]:
NUM_THREADS = 8
threaded_website_uptimer(WEBSITE_LIST, NUM_THREADS)

Took 0.7726471424102783s


In [21]:
%timeit -r 2 -n 5 threaded_website_uptimer(WEBSITE_LIST, 8)

Took 0.803419828414917s


Took 0.7941670417785645s


Took 0.7443621158599854s


Took 0.792698860168457s


Took 0.8574557304382324s


Took 0.7735133171081543s


Took 1.3289809226989746s


Took 0.7256917953491211s


Took 0.7036731243133545s
Took 0.718595027923584s
824 ms ± 25.9 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [22]:
%timeit -r 2 -n 5 multiprocess_website_uptimer(WEBSITE_LIST, 8)

The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 0.8968989849090576s


The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 0.8091881275177002s


The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.029736042022705s


The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 0.9217569828033447s
The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down


Took 0.9243528842926025s


...

The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.2697200775146484s


The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 0.8762547969818115s


The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down
Took 1.252098798751831s


The websie http://stackoverflow.com is down
The websie http://really-cool-available-domain.com is down
The websie http://another-really-interesting-domain.co is down


The websie http://baidu.com is down
Took 1.2650649547576904s
1.02 s ± 100 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
